## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
import os
import keras
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # 使用 CPU


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot = OneHotEncoder(categories='auto')
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:
classifier = Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3), input_shape=(32,32,3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3,3)))
classifier.add(BatchNormalization())

classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(units=10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(x_train,y_train, batch_size=100, epochs=100)

Epoch 1/100
50000/50000 [==============================] - 175s 4ms/step - loss: 1.4556 - accuracy: 0.4903
Epoch 2/100
50000/50000 [==============================] - 173s 3ms/step - loss: 1.0442 - accuracy: 0.6328
Epoch 3/100
50000/50000 [==============================] - 161s 3ms/step - loss: 0.9048 - accuracy: 0.6839
Epoch 4/100
50000/50000 [==============================] - 169s 3ms/step - loss: 0.8085 - accuracy: 0.7182
Epoch 5/100
50000/50000 [==============================] - 168s 3ms/step - loss: 0.7234 - accuracy: 0.7484
Epoch 6/100
50000/50000 [==============================] - 168s 3ms/step - loss: 0.6432 - accuracy: 0.7755
Epoch 7/100
50000/50000 [==============================] - 169s 3ms/step - loss: 0.5690 - accuracy: 0.8024
Epoch 8/100
50000/50000 [==============================] - 171s 3ms/step - loss: 0.4940 - accuracy: 0.8293
Epoch 9/100
50000/50000 [==============================] - 170s 3ms/step - loss: 0.4194 - accuracy: 0.8542
Epoch 10/100
50000/50000 [===========

50000/50000 [==============================] - 104s 2ms/step - loss: 0.0214 - accuracy: 0.9936
Epoch 77/100
50000/50000 [==============================] - 104s 2ms/step - loss: 0.0191 - accuracy: 0.9934
Epoch 78/100
50000/50000 [==============================] - 104s 2ms/step - loss: 0.0221 - accuracy: 0.9927
Epoch 79/100
50000/50000 [==============================] - 103s 2ms/step - loss: 0.0195 - accuracy: 0.9935
Epoch 80/100
50000/50000 [==============================] - 104s 2ms/step - loss: 0.0214 - accuracy: 0.9931
Epoch 81/100
50000/50000 [==============================] - 105s 2ms/step - loss: 0.0281 - accuracy: 0.9911
Epoch 82/100
50000/50000 [==============================] - 104s 2ms/step - loss: 0.0232 - accuracy: 0.9920
Epoch 83/100
50000/50000 [==============================] - 104s 2ms/step - loss: 0.0165 - accuracy: 0.9949
Epoch 84/100
50000/50000 [==============================] - 104s 2ms/step - loss: 0.0121 - accuracy: 0.9964
Epoch 85/100
50000/50000 [===============

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.5961840e-10, 0.0000000e+00, 1.3393174e-15, 1.0000000e+00,
        9.1565460e-13, 8.4966551e-35, 0.0000000e+00, 0.0000000e+00,
        6.4188774e-34, 0.0000000e+00]], dtype=float32)

In [7]:
from keras.layers import GlobalAveragePooling2D
# 嘗試不同搭法
classifier = Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3), input_shape=(32,32,3), padding='same', activation='relu'))
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(64, (3,3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#GAP
classifier.add(GlobalAveragePooling2D())

#FC
classifier.add(Dense(units=100, activation='relu'))

#輸出
classifier.add(Dense(units=10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.fit(x_train,y_train, batch_size=100, epochs=100)

input_example = (np.zeros(shape=(1,32,32,3)) - mean_train) / (std_train+1e-7) 
classifier.predict(input_example)

Epoch 1/100
50000/50000 [==============================] - 116s 2ms/step - loss: 1.5669 - accuracy: 0.4290
Epoch 2/100
50000/50000 [==============================] - 116s 2ms/step - loss: 1.2708 - accuracy: 0.5456
Epoch 3/100
50000/50000 [==============================] - 116s 2ms/step - loss: 1.1467 - accuracy: 0.5896
Epoch 4/100
50000/50000 [==============================] - 116s 2ms/step - loss: 1.0693 - accuracy: 0.6226
Epoch 5/100
50000/50000 [==============================] - 116s 2ms/step - loss: 1.0158 - accuracy: 0.6422
Epoch 6/100
50000/50000 [==============================] - 116s 2ms/step - loss: 0.9669 - accuracy: 0.6586
Epoch 7/100
50000/50000 [==============================] - 115s 2ms/step - loss: 0.9358 - accuracy: 0.6696
Epoch 8/100
50000/50000 [==============================] - 115s 2ms/step - loss: 0.9046 - accuracy: 0.6827
Epoch 9/100
50000/50000 [==============================] - 115s 2ms/step - loss: 0.8791 - accuracy: 0.6904
Epoch 10/100
50000/50000 [===========

50000/50000 [==============================] - 115s 2ms/step - loss: 0.4604 - accuracy: 0.8364
Epoch 77/100
50000/50000 [==============================] - 115s 2ms/step - loss: 0.4607 - accuracy: 0.8353
Epoch 78/100
50000/50000 [==============================] - 115s 2ms/step - loss: 0.4558 - accuracy: 0.8382
Epoch 79/100
50000/50000 [==============================] - 115s 2ms/step - loss: 0.4591 - accuracy: 0.8377
Epoch 80/100
50000/50000 [==============================] - 115s 2ms/step - loss: 0.4541 - accuracy: 0.8384
Epoch 81/100
50000/50000 [==============================] - 115s 2ms/step - loss: 0.4523 - accuracy: 0.8399
Epoch 82/100
50000/50000 [==============================] - 115s 2ms/step - loss: 0.4499 - accuracy: 0.8414
Epoch 83/100
50000/50000 [==============================] - 116s 2ms/step - loss: 0.4446 - accuracy: 0.8428
Epoch 84/100
50000/50000 [==============================] - 115s 2ms/step - loss: 0.4459 - accuracy: 0.8415
Epoch 85/100
50000/50000 [===============

array([[5.2294433e-03, 2.5378386e-08, 2.1823534e-01, 4.7234243e-01,
        1.8379028e-01, 2.5496907e-03, 6.2468164e-02, 4.6301760e-02,
        9.0821730e-03, 6.4089488e-07]], dtype=float32)